<a href="https://colab.research.google.com/github/prasann25/colab/blob/main/07_milestone_proj_1_food_vision_big.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MileStone Project 1 : Food Vision Big

See the annotated version of this notebook on GitHub : https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb

## Check GPU

Google Colab offers free GPUs(thank you Google), however, not all of them are compatible with mixed precision training. 
We need Nvidia V100, T4 or A100 to get the benefits of mixed precision training https://www.tensorflow.org/guide/mixed_precision

Google Colab offers :
* K80 (not compatible)
* P100 (not compatible)
* Tesla T4 (compatible)

Knowing this, in order to use mixed precision training, we need access to a Tesla T4( from within Google Colab) or if we're using our own hardware, our GPU needs a score of 7.0+ (see here : https://developer.nvidia.com/cuda-gpus).

In [1]:
! nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-45fdd78d-314a-67fb-6841-7913a94afcfa)


## Get helper functions
 In past modules, we've created a bunch of helper functions to do small tasks required for notebooks.

 Rather than rewrite all of these, we can import a script and load them in from there.
 The script we've got available can be found on GitHub :
 https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [2]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-07-30 05:28:29--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-07-30 05:28:29 (101 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
# Import series of helper functions for the notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## Use TensorFlow Datasets to Download Data

If you want to get an overview of TensorFlow Datasets (TFDS), read the guide :https://www.tensorflow.org/datasets/catalog/overview#all_datasets

In [4]:
# Get TensorFlow Datasets
import tensorflow_datasets as tfds


In [6]:
# List all available datasets
datasets_list = tfds.list_builders() # got all available datasets in TFDS
print("food101" in datasets_list) # is our taget dataset in the list of TFDS datasets

True


In [ ]:
#Load in the data(takes 5-6 minutes in Google Colab)
(train_data, test_data), ds_info = tfds.load(name="food101",
                                             split=["train", "validation"], 
                                             shuffle_files=True,
                                             as_supervised = True, # data gets returned in tuple format (data, label)
                                             with_info=True)
